# MDI+: Example Usages

In [1]:
%reload_ext autoreload
%autoreload 2
import sys
sys.path.append("../")
sys.path.append("../../")

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, mean_absolute_error, accuracy_score, roc_auc_score, mean_squared_error

from imodels.importance import RandomForestPlusRegressor, RandomForestPlusClassifier, \
    RidgeRegressorPPM, LassoRegressorPPM, IdentityTransformer
from imodels.importance.rf_plus import _fast_r2_score

In [2]:
def neg_mae(y_true, y_pred, **kwargs):
    """
    Evaluates negative mean absolute error
    """
    return -mean_absolute_error(y_true, y_pred, **kwargs)


def neg_log_loss(y_true, y_pred, epsilon=1e-15):
    """
    Evaluates negative log-loss
    """
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)  # Clip predicted probabilities to avoid extreme values
    return np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

In [3]:
# helper variables
rf_regressor = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, max_features=0.33, random_state=331)
rf_classifier = RandomForestClassifier(n_estimators=100, min_samples_leaf=1, max_features="sqrt", random_state=331)

## 1. Regression Example

In [4]:
# generate data from linear model: y = x1 + x2 + N(0, 1)
n = 200
p = 10
s = 2
X = np.random.normal(size=(n, p))
beta = np.concatenate((np.ones(s), np.zeros(p-s)))
y = np.matmul(X, beta) + np.random.normal(size=n)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=12345)

### 1.1 MDI+ with default settings for regression

In [5]:
# fit RF+
rf_plus_model = RandomForestPlusRegressor(rf_model=rf_regressor)
rf_plus_model.fit(X_train, y_train)

In [6]:
# make predictions with RF+
preds = rf_plus_model.predict(X_test)
r2_score(y_test, preds)

0.6236360279181548

In [7]:
# get MDI+ scores (higher r^2 value = greater importance)
mdi_plus_scores = rf_plus_model.get_mdi_plus_scores(X_train, y_train)
mdi_plus_scores.sort_values("importance", ascending=False)

,var,importance
1,1,0.279777
0,0,0.242784
5,5,-0.000699
2,2,-0.003691
7,7,-0.004302
4,4,-0.005231
3,3,-0.005823
8,8,-0.006085
9,9,-0.006333
6,6,-0.007457


#### 1.1.1. Local Feature Importances

In [8]:
# get global MDI+ scores using default scoring function (i.e., R^2 for regression so higher = greater importance)
# and local MDI+ scores using mean squared error (so lower = greater importance)
mdi_plus_scores = rf_plus_model.get_mdi_plus_scores(
    X_train, y_train, local_scoring_fns=mean_squared_error
)

In [9]:
mdi_plus_scores["global"].sort_values("importance", ascending=False)

,var,importance
1,1,0.279777
0,0,0.242784
5,5,-0.000699
2,2,-0.003691
7,7,-0.004302
4,4,-0.005231
3,3,-0.005823
8,8,-0.006085
9,9,-0.006333
6,6,-0.007457


In [10]:
mdi_plus_scores["local"]

,0,1,2,3,4,5,6,7,8,9
0,1.441626,1.109556,0.106475,0.072105,0.072478,0.062182,0.094234,0.072579,0.076139,0.082780
1,0.104441,1.962484,0.585299,0.449452,0.427229,0.566528,0.492860,0.497481,0.525555,0.462174
2,1.322728,0.140525,0.007655,0.010769,0.012496,0.014245,0.010718,0.015243,0.010803,0.007628
3,3.595042,0.807844,3.934170,3.768775,3.846275,3.715765,3.704057,4.025454,3.741831,3.878774
4,0.311039,2.280662,0.681646,0.817150,0.683187,0.762547,0.779904,0.722771,0.737181,0.704091
...,...,...,...,...,...,...,...,...,...,...
129,1.512446,3.055195,3.276597,3.112024,3.084210,3.138752,2.946486,3.122493,3.058612,3.048397
130,6.308365,0.170479,1.672127,1.683363,1.902355,1.764894,1.699032,1.722404,1.628380,1.630559
131,0.430185,0.623461,0.323257,0.273832,0.297575,0.269615,0.316479,0.265710,0.294890,0.316750
132,0.632103,2.486750,3.239558,3.118425,3.217633,3.098267,3.188929,3.234352,3.082505,3.369401


### 1.2 MDI+ with custom partial prediction model and evaluation metric(s)

In [11]:
# fit RF+ with custom partial prediction model
rf_plus_model = RandomForestPlusRegressor(rf_model=rf_regressor, prediction_model=LassoRegressorPPM())
rf_plus_model.fit(X_train, y_train)

In [12]:
# get MDI+ scores with custom evaluation metrics/scorers
mdi_plus_scores = rf_plus_model.get_mdi_plus_scores(X_train, y_train, scoring_fns={"r2_score": _fast_r2_score, "negative_mae": neg_mae})
mdi_plus_scores.sort_values("r2_score", ascending=False)

,var,r2_score,negative_mae
1,1,0.286972,-1.096935
0,0,0.245260,-1.133246
7,7,-0.002334,-1.338106
5,5,-0.002738,-1.337874
2,2,-0.002837,-1.339012
3,3,-0.003279,-1.337707
8,8,-0.003511,-1.337891
6,6,-0.003702,-1.339329
9,9,-0.004236,-1.337849
4,4,-0.004788,-1.339245


#### 1.2.1. Local Feature Importances

In [13]:
# get global and local MDI+ scores with custom evaluation metrics/scorers
mdi_plus_scores = rf_plus_model.get_mdi_plus_scores(
    X_train, y_train,
    scoring_fns={"r2_score": _fast_r2_score, "negative_mae": neg_mae},
    local_scoring_fns=True
)

../imodels/importance/rf_plus.py:478: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - numerator / denominator
../imodels/importance/rf_plus.py:478: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - numerator / denominator
../imodels/importance/rf_plus.py:478: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - numerator / denominator
../imodels/importance/rf_plus.py:478: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - numerator / denominator
../imodels/importance/rf_plus.py:478: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - numerator / denominator
../imodels/importance/rf_plus.py:478: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - numerator / denominator
../imodels/importance/rf_plus.py:478: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - numerator / denominator
../imodels/importance/rf_plus.py:478: RuntimeWar

In [14]:
mdi_plus_scores["global"].sort_values("negative_mae", ascending=False)

,var,r2_score,negative_mae
1,1,0.286972,-1.096935
0,0,0.245260,-1.133246
3,3,-0.003279,-1.337707
9,9,-0.004236,-1.337849
5,5,-0.002738,-1.337874
8,8,-0.003511,-1.337891
7,7,-0.002334,-1.338106
2,2,-0.002837,-1.339012
4,4,-0.004788,-1.339245
6,6,-0.003702,-1.339329


In [15]:
mdi_plus_scores["local"]["negative_mae"]

,0,1,2,3,4,5,6,7,8,9
0,-1.218195,-1.086275,-0.263831,-0.251098,-0.259852,-0.250144,-0.277268,-0.250440,-0.258174,-0.263648
1,-0.324760,-1.430624,-0.718211,-0.683088,-0.678600,-0.708994,-0.692358,-0.699455,-0.703173,-0.681782
2,-1.097948,-0.353872,-0.061290,-0.060664,-0.066387,-0.071954,-0.067811,-0.080054,-0.066976,-0.059199
3,-1.911652,-0.852492,-1.977808,-1.973486,-1.971479,-1.958383,-1.946350,-1.989918,-1.952441,-1.972665
4,-0.500609,-1.530723,-0.849811,-0.879037,-0.860548,-0.874949,-0.871702,-0.852423,-0.861295,-0.844654
...,...,...,...,...,...,...,...,...,...,...
129,-1.232509,-1.740134,-1.780201,-1.760946,-1.753791,-1.754896,-1.723995,-1.762538,-1.755148,-1.748768
130,-2.522394,-0.355162,-1.282015,-1.286160,-1.302432,-1.297216,-1.286143,-1.294596,-1.280140,-1.279292
131,-0.627356,-0.815604,-0.555578,-0.531252,-0.537819,-0.526478,-0.541340,-0.521345,-0.538163,-0.553964
132,-0.722513,-1.562450,-1.792277,-1.775237,-1.788320,-1.767218,-1.785879,-1.790950,-1.772221,-1.802777


### 1.3 MDI+ with custom transformer

The example below is equivalent to running RF+ with `include_raw=True`

In [16]:
# fit RF+ with custom transformer
rf_plus_model = RandomForestPlusRegressor(rf_model=rf_regressor, include_raw=False, add_transformers=[IdentityTransformer()])
rf_plus_model.fit(X_train, y_train)

In [17]:
# get MDI+ scores
mdi_plus_scores = rf_plus_model.get_mdi_plus_scores(X_train, y_train)
mdi_plus_scores.sort_values("importance", ascending=False)

,var,importance
1,1,0.283971
0,0,0.244754
5,5,-0.000707
2,2,-0.004586
7,7,-0.004675
4,4,-0.005303
3,3,-0.005881
9,9,-0.006766
8,8,-0.006979
6,6,-0.008007


### 1.4 Choosing the GLM and scoring metric via stability score

There are many choices of GLMs and scoring metrics that can be made within the MDI+ framework.

One way to select the GLM and scoring metric in MDI+ is by evaluating the stability of the feature importances/rankings for each choice of GLM/metric and taking the GLM/metric that is the most stable across different bootstrap samples of trees. For example, we can take the GLM and metric with the highest stability score, as measured by RBO below.

In [18]:
n_bootstraps = 25
prediction_models = {"ridge": RidgeRegressorPPM(), "lasso": LassoRegressorPPM()}
scoring_fns = {"r2": _fast_r2_score, "neg_mae": neg_mae}
stability_dict = {}
for model_name, prediction_model in prediction_models.items():
    # fit RF+
    rf_plus_model = RandomForestPlusRegressor(rf_model=rf_regressor, prediction_model=prediction_model)
    rf_plus_model.fit(X_train, y_train)
    # get MDI+ scores
    mdi_plus_scores = rf_plus_model.get_mdi_plus_scores(X_train, y_train, scoring_fns=scoring_fns)
    # get MDI+ stability scores
    mdi_plus_stability_scores = rf_plus_model.get_mdi_plus_stability_scores(B=n_bootstraps)
    stability_dict[model_name] = mdi_plus_stability_scores

In [19]:
pd.concat(stability_dict, axis=0).reset_index().rename(columns={"level_0": "ppm"}).drop(columns=["level_1"]).sort_values("RBO", ascending=False)

,ppm,scorer,RBO,tauAP
0,ridge,r2,0.895594,0.795996
1,ridge,neg_mae,0.834894,0.740198
2,lasso,r2,0.812896,0.700536
3,lasso,neg_mae,0.779714,0.652245


### 1.5 Aggregating multiple MDI+ rankings in an ensemble

Instead of choosing a single GLM and metric to use in MDI+, it may be preferable in some cases to aggregate MDI+ feature importances/rankings across multiple choices of GLMs and metrics.

One naive method for doing this ensembling is to take the median rank across each choice of GLM and metric (as shown below). However, more creative ensembling schemes can also be explored.

In [20]:
prediction_models = {"ridge": RidgeRegressorPPM(), "lasso": LassoRegressorPPM()}
scoring_fns = {"r2": _fast_r2_score, "neg_mae": neg_mae}
mdi_plus_scores_dict = {}
for model_name, prediction_model in prediction_models.items():
    # fit RF+
    rf_plus_model = RandomForestPlusRegressor(rf_model=rf_regressor, prediction_model=prediction_model)
    rf_plus_model.fit(X_train, y_train)
    # get MDI+ scores
    mdi_plus_scores = rf_plus_model.get_mdi_plus_scores(X_train, y_train, scoring_fns=scoring_fns)
    for col in mdi_plus_scores.columns:
        if col != "var":
            mdi_plus_scores = mdi_plus_scores.rename(columns={col: model_name + "_" + col})
    mdi_plus_scores_dict[model_name] = mdi_plus_scores

In [21]:
mdi_plus_scores_df = pd.concat([df.set_index('var') for df in mdi_plus_scores_dict.values()], axis=1)
mdi_plus_ranks_df = mdi_plus_scores_df.rank(ascending=False).median(axis=1)
mdi_plus_ranks_df = pd.DataFrame(mdi_plus_ranks_df, columns=["median_rank"]).reset_index()
mdi_plus_ranks_df.sort_values("median_rank")

,var,median_rank
1,1,1.0
0,0,2.0
5,5,3.5
3,3,6.0
7,7,6.0
2,2,6.5
8,8,6.5
9,9,7.0
4,4,8.0
6,6,10.0


## 2. Classification Example

In [22]:
# generate data from logistic model: logit(E[Y|X]) = x1 + x2
n = 200
p = 10
s = 2
X = np.random.normal(size=(n, p))
beta = np.concatenate((np.ones(s), np.zeros(p-s)))
probs = 1 / (1 + np.exp(-np.matmul(X, beta)))
y = (np.random.uniform(size=n) < probs) * 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=12345)

### 2.1 MDI+ with default classification settings

In [23]:
# fit RF+
rf_plus_model = RandomForestPlusClassifier(rf_model=rf_classifier)
rf_plus_model.fit(X_train, y_train)

In [24]:
# make predictions with RF+
preds = rf_plus_model.predict(X_test)
prob_preds = rf_plus_model.predict_proba(X_test)
accuracy_score(y_test, preds), roc_auc_score(y_test, prob_preds[:, 1])

(0.7575757575757576, 0.8787593984962406)

In [25]:
# get MDI+ scores (higher ngative log-loss value = greater importance)
mdi_plus_scores = rf_plus_model.get_mdi_plus_scores(X_train, y_train)
mdi_plus_scores.sort_values("importance", ascending=False)

,var,importance
0,0,-0.611236
1,1,-0.619968
8,8,-0.682693
7,7,-0.694262
4,4,-0.694657
5,5,-0.695846
2,2,-0.697825
3,3,-0.697842
6,6,-0.697853
9,9,-0.697854


#### 2.1.1. Local Feature Importances

In [26]:
# get MDI+ scores (higher ngative log-loss value = greater importance)
mdi_plus_scores = rf_plus_model.get_mdi_plus_scores(
    X_train, y_train, local_scoring_fns=neg_log_loss
)

In [27]:
mdi_plus_scores["global"].sort_values("importance", ascending=False)

,var,importance
0,0,-0.611236
1,1,-0.619968
8,8,-0.682693
7,7,-0.694262
4,4,-0.694657
5,5,-0.695846
2,2,-0.697825
3,3,-0.697842
6,6,-0.697853
9,9,-0.697854


In [28]:
mdi_plus_scores["local"]

,0,1,2,3,4,5,6,7,8,9
0,-0.216479,-1.172916,-0.470820,-0.545394,-0.576718,-0.548130,-0.571722,-0.498400,-0.557418,-0.566462
1,-0.587499,-0.503095,-0.833938,-0.798727,-0.814780,-0.798016,-0.866046,-0.831395,-0.746586,-0.789955
2,-0.808003,-0.363758,-0.830201,-0.869650,-0.764886,-0.812473,-0.808661,-0.815958,-0.707285,-0.877089
3,-0.255469,-0.448475,-0.461606,-0.554558,-0.592228,-0.617331,-0.542109,-0.528450,-0.484905,-0.567123
4,-1.546905,-0.198936,-0.742055,-0.865773,-0.778822,-1.062021,-0.777047,-0.971943,-0.906873,-0.877860
...,...,...,...,...,...,...,...,...,...,...
129,-0.773690,-0.221940,-0.608869,-0.569908,-0.566074,-0.545340,-0.548790,-0.558634,-0.549269,-0.635422
130,-1.470868,-0.619016,-0.884725,-0.897923,-0.798674,-0.831613,-0.896996,-0.779019,-0.941617,-1.001155
131,-1.571851,-0.698116,-0.521071,-0.551166,-0.580817,-0.588926,-0.571167,-0.588543,-0.436418,-0.553002
132,-0.203678,-0.306391,-0.990438,-0.857359,-0.805015,-0.812245,-0.850179,-0.853869,-0.945000,-0.854497


Note: `neg_log_loss()` was re-defined here since the default sklearn `log_loss()` metric requires vector inputs and thus cannot be directly used as a local scoring function.